In [36]:
#Get a pickled dict from python and export it to json format.
import pickle
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show, output_file
from bokeh.layouts import row,column,layout ,widgetbox
from bokeh.models import ColumnDataSource,HoverTool,BoxSelectTool 
from bokeh.models.widgets import Button

from bokeh.models.callbacks import CustomJS
from bokeh.io import curdoc,curstate

from bokeh import events

In [37]:
from bokeh.io import output_notebook, push_notebook
output_notebook()

Loading BokehJS ...

In [24]:
output=pickle.load(open('Results/output_large.pkl','rb'))

In [5]:
print(output.keys())
lstm_output=output['lstm_output']
output['y_pred']=list(map(lambda x:float(x),output['y_pred']))
output['y_val']=list(map(lambda x:float(x),output['y_val']))

names=["dim"+str(i) for i in range(len(lstm_output[0]))]
df=pd.DataFrame(lstm_output,columns=names)
df['y_pred']=output['y_pred']
df['y_val']=output['y_val']
df.head()

dict_keys(['lstm_output', 'y_pred', 'y_val'])


,dim0,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,...,dim120,dim121,dim122,dim123,dim124,dim125,dim126,dim127,y_pred,y_val
0,0.001742,-0.043197,-0.004003,-0.665576,0.307256,0.014742,0.082608,0.104317,-0.000000,-0.049952,...,0.125784,0.028014,-0.099455,-0.187336,0.012104,0.378489,0.185662,-0.636480,3.0,3.0
1,0.001588,-0.095552,0.000000,0.001404,0.419409,0.238556,0.038814,0.114610,-0.035958,-0.576853,...,-0.004567,0.178218,0.012994,-0.032227,0.074585,0.376468,-0.013098,-0.289579,4.0,4.0
2,0.009536,-0.062648,-0.010760,-0.062144,0.059464,0.087761,0.030383,-0.227709,-0.000000,-0.035097,...,-0.061632,-0.400253,-0.292240,-0.031306,-0.064614,0.714171,0.035908,-0.427725,3.0,3.0
3,0.000000,0.240674,-0.000000,-0.626155,0.332174,0.000000,0.406524,0.364807,-0.000000,-0.089410,...,-0.261884,0.130471,-0.084853,0.169132,-0.224658,0.796961,-0.017889,-0.090232,4.0,3.0
4,0.100477,-0.060124,0.032336,0.098073,-0.004858,0.355356,0.091956,-0.147874,-0.000000,0.249866,...,0.012617,0.421778,-0.023826,-0.129779,0.139328,0.333764,0.230833,-0.830991,3.0,3.0


In [6]:
import json 
df.to_json('sentiment_large.json',orient='records')

with open('sentiment.json','w') as fp:
     json.dump(df.as_matrix().tolist(),fp) 
        
df2=df#.iloc[:10,:]
df2.to_csv('sentiment_large2.csv')        

In [9]:
losses=pickle.load(open('Results/summary_losses.pkl','rb'))
epoch_losses=losses['epoch_losses']
batch_losses=losses['batch_losses']

elosses_df=pd.DataFrame(data=dict(epoch_losses=epoch_losses))
elosses_df['epoch_number']=list(range(len(epoch_losses)))
elosses_df.to_csv('./Results/summary_epoch_losses.csv')

blosses_df=pd.DataFrame(data=dict(batch_losses=batch_losses))
blosses_df['batch_number']=list(range(len(batch_losses)))
blosses_df.to_csv('./Results/summary_batch_losses.csv')

In [38]:
source=ColumnDataSource(data=dict(x=elosses_df['epoch_number'],y=elosses_df['epoch_losses']))
source2=ColumnDataSource(elosses_df)

source3=ColumnDataSource(data=dict(x=blosses_df['batch_number'],y=blosses_df['batch_losses']))
source4=ColumnDataSource(blosses_df)
print(source2.column_names,source4.column_names)

#Configure box-select tool
bs_callback = CustomJS(args=dict(source=source,source2=source2),code="""

        var geometry = cb_data['geometry'];
        var data = source['data'];
        var data2=source2['data'];
        
        /// calculate Rect attributes
        var x1 = Math.floor(geometry['x1']) ;
        var x0 = Math.floor(geometry['x0']);
        var height = geometry['y1'] - geometry['y0'];
        
        function is_within_window(el){
          return (el>x0 && el<x1);
        } 
    
        function is_index_within_window(el){
            var idx=data2['epoch_losses'].indexOf(el);
            return is_within_window(idx);
        }
        
        data['x']=data2['epoch_number'].filter(is_within_window) ;
        data['y']=data2['epoch_losses'].filter(is_index_within_window);
        source.trigger('change');
        console.log(x1,x0); 

    """)

bs_callback2 = CustomJS(args=dict(source=source3,source2=source4),code="""

        var geometry = cb_data['geometry'];
        var data = source['data'];
        var data2=source2['data'];
        
        /// calculate Rect attributes
        var x1 = Math.floor(geometry['x1']) ;
        var x0 = Math.floor(geometry['x0']);
        var height = geometry['y1'] - geometry['y0'];
        
        function is_within_window(el){
          return (el>x0 && el<x1);
        } 
    
        function is_index_within_window(el){
            var idx=data2['batch_losses'].indexOf(el);
            return is_within_window(idx);
        }
        
        data['x']=data2['batch_number'].filter(is_within_window) ;
        data['y']=data2['batch_losses'].filter(is_index_within_window);
        source.trigger('change');
        console.log(x1,x0); 

    """)

rs_callback = CustomJS(args=dict(source=source,source2=source2),code="""

        var geometry = cb_data['geometry'];
        var data = source['data'];
        var data2=source2['data'];
        
    
        data['x']=data2['epoch_number'];
        data['y']=data2['epoch_losses'];
        source.trigger('change');
        console.log(x1,x0); 

    """)

rs_callback2 = CustomJS(args=dict(source=source3,source2=source4),code="""

        var geometry = cb_data['geometry'];
        var data = source['data'];
        var data2=source2['data'];
        
    
        data['x']=data2['batch_number'];
        data['y']=data2['batch_losses'];
        source.trigger('change');
        console.log(x1,x0); 

    """)
box_select=BoxSelectTool(callback=bs_callback)
box_select2=BoxSelectTool(callback=bs_callback2)

button1=Button(label='Reset')
button1.js_on_event(events.ButtonClick,rs_callback) 

button2=Button(label='Reset')
button2.js_on_event(events.ButtonClick, rs_callback2) 

hover1=HoverTool(tooltips=[('Epoch Number:',"@x"),\
                          ('Epoch Loss:','@y')],show_arrow=False)

plot1 = figure(plot_width=400, plot_height=400,x_axis_label='epoch_number',y_axis_label='epoch_loss',\
                tools=[box_select])
plot1.add_tools(hover1)

plot1.line('x','y',line_width=2,source=source)

source2=ColumnDataSource(blosses_df)
hover2=HoverTool(tooltips=[('Batch Number:',"@x"),\
                          ('Batch Loss:','@y')],show_arrow=False)
plot2 = figure(plot_width=600, plot_height=400,x_axis_label='batch_number',y_axis_label='batch_loss',\
              tools=[box_select2])
plot2.add_tools(hover2)
plot2.line('x','y',line_width=2,color='green',source=source3)

layout=row(column(plot1,widgetbox(button1)),column(plot2,widgetbox(button2)))
show(layout)
curdoc().add_root(layout)

['epoch_losses', 'epoch_number', 'index'] ['batch_losses', 'batch_number', 'index']


In [8]:
preds=[]
vals=[]
with open('./Results/summarization_results.txt','r') as fd:
    while(True):
        line=fd.readline()
        if(line==''):
            break
        if("y_pred:" in line):
            preds.append(line.strip().split(':')[1].strip())
        else:
            vals.append(line.strip().split(':')[1].strip())

In [28]:
from __future__ import division
from collections import Counter,defaultdict
#this is an implementation of unigram bleu score.
def unigram_bleu_score(pred_list,val_list):
    score=0
    N=0
    for i in range(len(pred_list)):
        pred=pred_list[i].split()
        val=val_list[i].split()
        c_pred=Counter(pred)
        c_val=Counter(val)
        N+=len(c_val.keys())
        for key,value in c_pred.items():
            score+=min(value,c_val[key])
    print("Score: ",score,"N: ",N)        
    bleu_score=score/N
    return bleu_score

def bigram_bleu_score(pred_list,val_list):
          score=0
          N=0
          for i in range(len(pred_list)):
              pred=pred_list[i].split()
              val=val_list[i].split()
            
              pred=list(zip(pred[:-1],pred[1:]))
              val=list(zip(val[:-1],val[1:]))
              c_pred=Counter(pred)
              c_val=Counter(val)
              N+=len(c_val.keys())
              for key,value in c_pred.items():
                  score+=min(value,c_val[key])
          print("Score: ",score,"N: ",N)
          bleu_score=score/N
          return bleu_score        

In [29]:
bleu_score=unigram_bleu_score(preds,vals)
print(bleu_score)
bleu_score=bigram_bleu_score(preds,vals)
print(bleu_score)

Score:  11836 N:  28662
0.41295094550275624
Score:  2280 N:  23320
0.09777015437392796


In [ ]:
#plot the results in scatter plot using bokeh.
#the circle sizes depend on how many matches in each sentence.
